<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install 
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

# import functions/Classes for metrics
from pyspark.ml.evaluation import RegressionEvaluator


In [3]:
# Start session

In [5]:
spark= SparkSession.builder.appName('ProjectSpark').getOrCreate()

25/04/30 10:10:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

data=spark.read.csv('searchterms.csv',header=True,inferSchema=True)
data.printSchema()

--2025-04-30 10:11:00--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.4’

searchterms.csv.4   100%[===================>] 227.99K  --.-KB/s    in 0.004s  

2025-04-30 10:11:00 (55.9 MB/s) - ‘searchterms.csv.4’ saved [233457/233457]

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [7]:
# Load the csv into a spark dataframe

In [9]:
data=spark.read.csv('searchterms.csv',header=True,inferSchema=True)
data.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [10]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [11]:
print(f'Number of rows:{data.count()}')
print(f'Number of columns:{len(data.columns)}')

Number of rows:10000
Number of columns:4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [12]:
data.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [14]:
print(data.schema['searchterm'].dataType)

StringType


In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [25]:
#data.groupBy('searchterm').count().show()
#data.groupBy('searchterm').count().where(data.searchterm == 'laptop 14 inch').show()

from pyspark.sql.functions import col,desc
data.groupBy('searchterm').count().filter(col('searchterm') == 'laptop 14 inch').show()

+--------------+-----+
|    searchterm|count|
+--------------+-----+
|laptop 14 inch|  461|
+--------------+-----+



In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [27]:
data.groupBy('searchterm').count().orderBy(col('count').desc()).show(5)

[Stage 42:=============================================>       (171 + 12) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [29]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2025-04-30 10:30:19--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2025-04-30 10:30:19 (13.2 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [30]:
!tar -xzvf model.tar.gz

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
sales_prediction.model/data/._SUCCESS.crc


In [36]:
from pyspark.ml.regression import LinearRegressionModel
# Load the model
model = LinearRegressionModel.load('./sales_prediction.model')

In [37]:
model.params

[Param(parent='LinearRegression_6d5736f3dbe7', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='featuresCol', doc='features column name'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='fitIntercept', doc='whether to fit an intercept term'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='labelCol', doc='label column name'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='maxIter', doc

In [40]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

df1 = spark.read.parquet('./sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet')
df1.printSchema()

root
 |-- intercept: double (nullable = true)
 |-- coefficients: vector (nullable = true)
 |-- scale: double (nullable = true)



In [41]:
df1.head()

Row(intercept=-13019.989140447298, coefficients=DenseVector([6.5226]), scale=1.0)

In [44]:
df1.head()[0]

-13019.989140447298

In [58]:
def predict(year):
    assembler=VectorAssembler(inputCols=['year'],outputCol='features')
    data=[[year,0]]
    columns=["year","sales"]
    df1=spark.createDataFrame(data,columns)
    df2=assembler.transform(df1).select('features','sales')
    prediction=model.transform(df2)
    prediction.select('prediction').show()

In [59]:
    data=[['2023',0]]
    columns=["year","sales"]
    df1=spark.createDataFrame(data,columns)
    df1.show()

+----+-----+
|year|sales|
+----+-----+
|2023|    0|
+----+-----+



In [60]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+

